In [2]:
import os
import re
import upload_v2 as v2

In [3]:
dict_refer = v2.dict_refer
comp = v2.comp

In [3]:
files = ["20422648 S36BW-822041113360_(1) 등초본.pdf"]

for f in files:
    name_items = {"key" :"", "name" :"", "docu" :"", "event" :"", "extra" :"", "date" :""}
    name_extra = None
    n = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]

    name_items["key"], n = v2.keyFnc(n)
    print("afterkeyF n = ", n)
    isEvent, docuEventNameextra = v2.eventFnc(n) ### docu, event
    print("isEvent", isEvent)
    if isEvent == 2 :
        print("경정사건인데 nodocu")
        continue
    elif isEvent == 1 : # 정상
        name_items["docu"] = docuEventNameextra[0] #사건번호>>docu 할당 #############
        name_items["event"] = docuEventNameextra[1]
        name_extra = docuEventNameextra[2]
    else : # 사건번호 없음
        isDocu, docuNameExtra = v2.setDocu(n)
        print("isDocu : ", isDocu, "docuNameExtra : ", docuNameExtra)
        if isDocu :
            name_items["docu"] = docuNameExtra[0]#비사건번호>>docu 할당 #############
            name_extra = docuNameExtra[1]
        else : # nodocu
            print("setDocu 결과 nodocu")
            continue
    date, temp = v2.dateFnc(name_extra) ### date, name, extra
    print(f"date : {date}, temp = {temp}")
    name, extra = v2.rmNeedlessSharp(temp)   
    print(f"name = {name}, extra={extra}")
    if len(name) < 2 : 
        try :
            name = dict_refer[name_items["key"]][2]
        except :
            print("dict_refer 키에러 for name")
        name = v2.rmNeedlessSharp(name+"#")[0] #list를 반환하기 때문에 
        name = v2.rm_s(name)

    name_items["name"] = name #### name, extra, date 할당 ######################
    name_items["extra"] = extra
    name_items["date"] = date
    new_f = "_".join(filter(lambda x :bool(x), name_items.values())) + ext
    print(f, ">>>", new_f, "-----------")

afterkeyF n =  S36BW-822041113360 (1) 등초본
isEvent 0
isDocu :  True docuNameExtra :  ['집행권원 재도', '#S36BW-822041113360 (1) 등초본']
date : , temp = #S36BW-822041113360 (1) 등초본
name = , extra=BW 등초본
20422648 S36BW-822041113360_(1) 등초본.pdf >>> 20422648_박영태_집행권원 재도_BW 등초본.pdf -----------


In [20]:
p_rmSerialN = re.compile(r"(?<![a-zA-Z])[a-zA-Z](?![a-zA-Z])|\d+[a-zA-Z]+|[a-zA-Z]+\d+|TAA\(회\)|\
        |SCSB|ADMIN.*Conflict|\d(?!건|통|차|채|억|천|백|급|번|길)") #3개의 숫자를 지운다. 해당글자가 나온다면 그 앞 숫자는 살린다. 
p_sign = re.compile(r"[^#㈜()\sa-zA-Zㄱ-ㅎ가-힣\d]|\([^w#]*\)") #반쪽 괄호만 있는 거는 어케 지우지?

nameextra = "204226a48 S36BW-822041113360_aa(1)a aa등초본.pdf"
nameextra = p_rmSerialN.sub("", nameextra)
nameextra = p_sign.sub(" ", nameextra)
nameextra

'   aa  aa등초본 pdf'

In [4]:
subStem = "ghdrlfehd 3월일 초본"
# p_y = re.compile(r'(\d\d|\d\d\d\d)년')
# p_m = re.compile(r'(\d{1,2})월')
# p_d = re.compile(r'(\d{1,2})일')
# if p_y.search(subStem) or p_m.search(subStem) or p_d.search(subStem) :
#     print("good")
#     d = ""
#     py = p_y.search(subStem)
#     pm = p_m.search(subStem)
#     pd = p_d.search(subStem)
#     if py : 
#         d = py[1]
#         subStem = p_y.sub("", subStem)
#     if pm :
#         if len(pm[1]) == 1 : d = d + "0" + pm[1]
#         else : d = d + pm[1]
#         subStem = p_m.sub("", subStem)
#     if pd :
#         if len(pd[1]) == 1 : d = d + "0" + pd[1]
#         else : d = d + pd[1]
#         subStem = p_d.sub("", subStem)
#         print("yes", d, subStem)
v2.dateFnc(subStem)

('03', 'ghdrlfehd 일 초본')

In [53]:
p=re.compile(r'\(?\s?법인\s?등기부\s?(등본)?\s?\)?|\(?\s?(법인)?\s?등[기|본]\s?\)?|\(?\s?주민\s?등록\s?(등본|정보)\s?\)?')
p.search("20433506 김상진 주민등록정보_(1).pdf")


<re.Match object; span=(12, 19), match=' 주민등록정보'>